In [1]:
import numpy as np
import pandas as pd
import random

from pathlib import Path
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.metrics import AUC
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, GRU, Input, BatchNormalization, Dropout
from ncps.wirings import AutoNCP
from ncps.keras import LTC

2025-05-03 00:50:01.978700: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-03 00:50:02.109482: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-03 00:50:02.238109: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746222602.334858    1645 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746222602.362695    1645 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-03 00:50:02.604309: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

# Configuration

In [2]:
NUM_EPOCHS = 400
NUM_EXPERIMENTS = 5

def create_model(train):
    model = Sequential()
    model.add(Input(shape=(train.shape[1], train.shape[2])))

    model.add(LTC(AutoNCP(64, 32), return_sequences=True))
    model.add(LTC(AutoNCP(32, 16), return_sequences=False))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=0.002, weight_decay=1e-7, use_ema=True), loss='binary_crossentropy', metrics=["accuracy", AUC(name="auc")])
    return model

# Experiment

In [3]:
ID = ["ID"]
USER = ["SubjectID"]
IDS = ["SubjectID", "VideoID"]
TARGET = ["predefinedlabel"]
FEATURES = ["Delta", "Theta", "Alpha1", "Alpha2", "Beta1", "Beta2", "Gamma1", "Gamma2"]
LAGS = [1]
INIT_SEED = 5412

In [4]:
data_dir = Path("/home/aseliverstov/projects/brain_signals/data")
data = pd.read_csv(data_dir / "EEG_data.csv")

new_features = []
for lag in LAGS:
    for feature_name in FEATURES:
        new_feature_name = f"{feature_name}_{lag}"
        new_features.append(new_feature_name)
        data[new_feature_name] = data.groupby(IDS)[feature_name].shift(lag).fillna(0)
FEATURES.extend(new_features)

data["ID"] = (len(np.unique(data["VideoID"])) * data["SubjectID"] + data["VideoID"]).astype("int")
data = data[ID + USER + FEATURES + TARGET]

data.head(3)

,ID,SubjectID,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2,Delta_1,Theta_1,Alpha1_1,Alpha2_1,Beta1_1,Beta2_1,Gamma1_1,Gamma2_1,predefinedlabel
0,0,0.0,301963.0,90612.0,33735.0,23991.0,27946.0,45097.0,33228.0,8293.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,73787.0,28083.0,1439.0,2240.0,2746.0,3687.0,5293.0,2740.0,301963.0,90612.0,33735.0,23991.0,27946.0,45097.0,33228.0,8293.0,0.0
2,0,0.0,758353.0,383745.0,201999.0,62107.0,36293.0,130536.0,57243.0,25354.0,73787.0,28083.0,1439.0,2240.0,2746.0,3687.0,5293.0,2740.0,0.0


In [5]:
def reshape_dataset(data):
    features = []
    target = []
    for cur_id in np.unique(data[ID].to_numpy()):
        cur_id_data = data[data[ID].to_numpy() == cur_id]
        target.append(np.mean(cur_id_data[TARGET].to_numpy()).astype("int"))
        features.append(cur_id_data[FEATURES].to_numpy())

    features = pad_sequences(features)
    return np.array(features), np.array(target)

def pad_sequences(arrays, pad_value=0):
    max_length = max(arr.shape[0] for arr in arrays)
    padded_arrays = [
        np.pad(
            arr,
            ((0, max_length - arr.shape[0]), (0, 0)),
            mode='constant',
            constant_values=pad_value)
            for arr in arrays
        ]
    return np.stack(padded_arrays)

In [6]:
X, _ = reshape_dataset(data)
model = create_model(X)
model.summary()

2025-05-03 00:50:06.294823: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ ltc (LTC)                       │ (None, 144, 32)        │        20,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ltc_1 (LTC)                     │ (None, 16)             │         8,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,169 (113.94 KB)

 Trainable params: 29,169 (113.94 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
%%time

all_acc = []
all_loss = []
all_auc = []

all_val_acc = []
all_val_loss = []
all_val_auc = []

for j, seed in tqdm(enumerate(np.arange(NUM_EXPERIMENTS) + INIT_SEED)):
    np.random.seed(int(seed))
    random.seed(int(seed))
    tf.random.set_seed(int(seed))

    train_id = np.random.choice(np.unique(np.ravel(data[USER])), 7, replace=False)
    train_index = np.isin(data[USER], train_id)

    train = data.iloc[train_index]
    test = data.iloc[~train_index]

    X_train, y_train = reshape_dataset(train)
    X_test, y_test = reshape_dataset(test)

    y_train = y_train.reshape(-1, 1)
    y_test = y_test.reshape(-1, 1)

    model = create_model(X_train)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=NUM_EPOCHS,
        batch_size=10,
        verbose=1,
    )

    acc = history.history['accuracy']
    loss = history.history['loss']
    auc = history.history['auc']

    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    val_auc = history.history['val_auc']

    all_acc.append(acc)
    all_loss.append(loss)
    all_auc.append(auc)

    all_val_acc.append(val_acc)
    all_val_loss.append(val_loss)
    all_val_auc.append(val_auc)

epoch_acc = np.mean(all_acc, axis=0)
epoch_loss = np.mean(all_loss, axis=0)
epoch_auc = np.mean(all_auc, axis=0)

epoch_val_acc = np.mean(all_val_acc, axis=0)
epoch_val_loss = np.mean(all_val_loss, axis=0)
epoch_val_auc = np.mean(all_val_auc, axis=0)

0it [00:00, ?it/s]

Epoch 1/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.5552 - auc: 0.5806 - loss: 0.6902 - val_accuracy: 0.6333 - val_auc: 0.8333 - val_loss: 0.6760
Epoch 2/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 894ms/step - accuracy: 0.6750 - auc: 0.7330 - loss: 0.6726 - val_accuracy: 0.8333 - val_auc: 0.8222 - val_loss: 0.6434
Epoch 3/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 907ms/step - accuracy: 0.7615 - auc: 0.7361 - loss: 0.6491 - val_accuracy: 0.8333 - val_auc: 0.8222 - val_loss: 0.6084
Epoch 4/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 944ms/step - accuracy: 0.7615 - auc: 0.7574 - loss: 0.6244 - val_accuracy: 0.8333 - val_auc: 0.7889 - val_loss: 0.5761
Epoch 5/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 915ms/step - accuracy: 0.7615 - auc: 0.7268 - loss: 0.6026 - val_accuracy: 0.8333 - val_auc: 0.7889 - val_loss: 0.5464
Epoch 6/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 874ms/step - accuracy: 0.7615 - auc: 0.7151 - loss: 0.5834 - val_accuracy: 0.8333 - val_auc: 0.7889 - val_loss: 0.5191
Epoch 7/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 949ms/st

1it [39:45, 2385.72s/it]

Epoch 1/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.4506 - auc: 0.5383 - loss: 0.7228 - val_accuracy: 0.5000 - val_auc: 0.8000 - val_loss: 0.6896
Epoch 2/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.5280 - auc: 0.6005 - loss: 0.6943 - val_accuracy: 0.7667 - val_auc: 0.7600 - val_loss: 0.6820
Epoch 3/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 934ms/step - accuracy: 0.6258 - auc: 0.7412 - loss: 0.6808 - val_accuracy: 0.7667 - val_auc: 0.7867 - val_loss: 0.6716
Epoch 4/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 914ms/step - accuracy: 0.7287 - auc: 0.7514 - loss: 0.6698 - val_accuracy: 0.8000 - val_auc: 0.7600 - val_loss: 0.6593
Epoch 5/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 996ms/step - accuracy: 0.7667 - auc: 0.7841 - loss: 0.6587 - val_accuracy: 0.8000 - val_auc: 0.7867 - val_loss: 0.6420
Epoch 6/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 945ms/step - accuracy: 0.7667 - auc: 0.7749 - loss: 0.6426 - val_accuracy: 0.8000 - val_auc: 0.7867 - val_loss: 0.6185
Epoch 7/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - a

2it [1:39:32, 3091.99s/it]

...

Epoch 1/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - accuracy: 0.4722 - auc: 0.3367 - loss: 0.7006 - val_accuracy: 0.5000 - val_auc: 0.8333 - val_loss: 0.6856
Epoch 2/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.7012 - auc: 0.8094 - loss: 0.6836 - val_accuracy: 0.8333 - val_auc: 0.8111 - val_loss: 0.6745
Epoch 3/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.8038 - auc: 0.8066 - loss: 0.6726 - val_accuracy: 0.8333 - val_auc: 0.8111 - val_loss: 0.6596
Epoch 4/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.8038 - auc: 0.7820 - loss: 0.6567 - val_accuracy: 0.8333 - val_auc: 0.8111 - val_loss: 0.6358
Epoch 5/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.8038 - auc: 0.7846 - loss: 0.6314 - val_accuracy: 0.8333 - val_auc: 0.8111 - val_loss: 0.6013
Epoch 6/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.8038 - auc: 0.7874 - loss: 0.5981 - val_accuracy: 0.8333 - val_auc: 0.8111 - val_loss: 0.5644
Epoch 7/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accurac

4it [4:29:20, 4442.92s/it]

Epoch 1/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - accuracy: 0.5346 - auc: 0.3260 - loss: 0.6990 - val_accuracy: 0.5000 - val_auc: 0.7733 - val_loss: 0.6784
Epoch 2/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.6917 - auc: 0.8917 - loss: 0.6572 - val_accuracy: 0.5000 - val_auc: 0.7733 - val_loss: 0.6375
Epoch 3/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.5404 - auc: 0.8256 - loss: 0.6186 - val_accuracy: 0.8000 - val_auc: 0.7467 - val_loss: 0.5958
Epoch 4/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.8754 - auc: 0.8158 - loss: 0.5639 - val_accuracy: 0.8000 - val_auc: 0.6800 - val_loss: 0.5654
Epoch 5/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.8754 - auc: 0.8281 - loss: 0.5223 - val_accuracy: 0.8000 - val_auc: 0.7467 - val_loss: 0.5370
Epoch 6/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.8600 - auc: 0.8089 - loss: 0.4866 - val_accuracy: 0.7333 - val_auc: 0.7467 - val_loss: 0.5159
Epoch 7/400
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accurac

5it [5:56:17, 4275.43s/it]

CPU times: user 11h 44min 34s, sys: 4h 52min 51s, total: 16h 37min 26s
Wall time: 5h 56min 17s


In [8]:
for i in range(NUM_EPOCHS):
    print(f"Epoch {(i + 1)}: TRAIN Accuracy = {np.round(epoch_acc[i], 3)} Loss = {np.round(epoch_loss[i], 3)} AUC = {np.round(epoch_auc[i], 3)}")
    print(f"Epoch {(i + 1)}: VAL Accuracy = {np.round(epoch_val_acc[i], 3)} Loss = {np.round(epoch_val_loss[i], 3)} AUC = {np.round(epoch_val_auc[i], 3)}")

with open("./logs/Liquidv3_emb1_expanded.txt", "w") as f:
    for i in range(NUM_EPOCHS):
        f.write(f"Epoch {(i + 1)}: TRAIN Accuracy = {np.round(epoch_acc[i], 3)} Loss = {np.round(epoch_loss[i], 3)} AUC = {np.round(epoch_auc[i], 3)}\n")
        f.write(f"Epoch {(i + 1)}: VAL Accuracy = {np.round(epoch_val_acc[i], 3)} Loss = {np.round(epoch_val_loss[i], 3)} AUC = {np.round(epoch_val_auc[i], 3)}\n")

Epoch 1: TRAIN Accuracy = 0.477 Loss = 0.699 AUC = 0.489
Epoch 1: VAL Accuracy = 0.527 Loss = 0.682 AUC = 0.81
Epoch 2: TRAIN Accuracy = 0.717 Loss = 0.673 AUC = 0.74
Epoch 2: VAL Accuracy = 0.753 Loss = 0.659 AUC = 0.796
Epoch 3: TRAIN Accuracy = 0.72 Loss = 0.652 AUC = 0.762
Epoch 3: VAL Accuracy = 0.813 Loss = 0.633 AUC = 0.796
Epoch 4: TRAIN Accuracy = 0.797 Loss = 0.626 AUC = 0.766
Epoch 4: VAL Accuracy = 0.82 Loss = 0.606 AUC = 0.77
Epoch 5: TRAIN Accuracy = 0.806 Loss = 0.599 AUC = 0.768
Epoch 5: VAL Accuracy = 0.82 Loss = 0.577 AUC = 0.784
Epoch 6: TRAIN Accuracy = 0.803 Loss = 0.571 AUC = 0.769
Epoch 6: VAL Accuracy = 0.807 Loss = 0.549 AUC = 0.784
Epoch 7: TRAIN Accuracy = 0.803 Loss = 0.545 AUC = 0.768
Epoch 7: VAL Accuracy = 0.82 Loss = 0.523 AUC = 0.779
Epoch 8: TRAIN Accuracy = 0.806 Loss = 0.522 AUC = 0.763
Epoch 8: VAL Accuracy = 0.82 Loss = 0.5 AUC = 0.779
Epoch 9: TRAIN Accuracy = 0.806 Loss = 0.503 AUC = 0.762
Epoch 9: VAL Accuracy = 0.82 Loss = 0.482 AUC = 0.779
Epo